In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import scipy.stats
from openpyxl import Workbook

In [2]:
#Read sensor data  version(1.0) 
def calibrateData_v1(axis,axislen):
    print("==calibrateData==")

    result = np.empty(shape=(axislen,9))
    temp = np.copy(axis[:])

    wear = 1
    
    if wear ==1:#LR
        for i in range(axislen):
            #取得正確值 && 導正軸位
            result[i, 0] = temp[i, 2] * 0.004          #acc_x單位G力
            result[i, 1] = temp[i, 0] * 0.004*(-1)     #acc_y 
            result[i, 2] = temp[i, 1] * 0.004*(-1)     #acc_z
            # 8 6 7
            result[i, 3] = temp[i, 8] * 0.803/3.33*(-1)  #gry_x單位角度
            result[i, 4] = temp[i, 6] * 0.803/3.33*(-1)  #gry_y
            result[i, 5] = temp[i, 7] * 0.803/3.33*(-1)  #gry_z
            result[i, 6] = temp[i, 5] / 768              #mag_x
            result[i, 7] = temp[i, 3] / 768              #mag_y
            result[i, 8] = temp[i, 4] / 768              #mag_z
    else:#W    
        for i in range(axislen):
            result[i, 0] = temp[i, 2] * 0.004           #acc_x單位G力
            result[i, 1] = temp[i, 0] * 0.004           #acc_y 
            result[i, 2] = temp[i, 1] * 0.004           #acc_z
            # 8 6 7
            result[i, 3] = temp[i, 8] * 0.803/3.33*(-1)  #gry_x單位角度
            result[i, 4] = temp[i, 6] * 0.803/3.33       #gry_y
            result[i, 5] = temp[i, 7] * 0.803/3.33       #gry_z
            result[i, 6] = temp[i, 5] / 768              #mag_x
            result[i, 7] = temp[i, 3] / 768              #mag_y
            result[i, 8] = temp[i, 4] / 768              #mag_z        
        
    for i in range(axislen):        
            result[i, 0] =  (2.0143) * result[i, 0] -0.1451;
            result[i, 1] =  (1.9083) * result[i, 1] -0.0248;
            result[i, 2] =  (1.8867) * result[i, 2] -0.0240;
        
        
    #(1.8850) * result[i, 2] -0.03864;(2.0007)* result[i, 1] + 0.0888;        
    calibrate =  np.copy(result);
    return calibrate

In [3]:
def setaxis(rawdata):
#do set axis
    print("==setaxis==")
    temp=np.empty(shape=(axislen,32))
    counter=0
    for i in range(axislen):
        for j in range(32):
            if counter<rawlen:
                temp[i,j] = rawdata[counter]
            counter=counter+1                
        
    #datasheet的軸位定義，之後要導正至自定義
    # eg:
    # sample accX accY accZ gryX gryY gryZ magX magY magZ
    # 28.  -56. -115.  389.  502. -164.   11.    7.   10.  
    # 28.  -55. -115.  377.  506. -172.    9.    7.    9.  
    
    #hex to dec
        axis[i,0] = temp[i,5]*256+temp[i,6]      #accX        
        axis[i,1] = temp[i,7]*256+temp[i,8]      #accY
        axis[i,2] = temp[i,9]*256+temp[i,10]     #accZ
        axis[i,3] = temp[i,11]*256+temp[i,12]    #gryX
        axis[i,4] = temp[i,13]*256+temp[i,14]    #gryY
        axis[i,5] = temp[i,15]*256+temp[i,16]    #gryZ
        axis[i,6] = temp[i,17]*256+temp[i,18]    #magX
        axis[i,7] = temp[i,19]*256+temp[i,20]    #magY
        axis[i,8] = temp[i,21]*256+temp[i,22]    #magZ
    #負數 eg. hex(101) -> dec(-2) 
    for i in range(axislen):
        for j in range(9):
            if axis[i,j]>32768:
                axis[i,j] = axis[i,j] - 65536
    
    return axis
#end do axis

In [4]:
# Data processing
# lowpass: N點取平均 
def lowpass(calibrate,axislen):
    print("==lowpass==")
    N = 20;

    temp = np.copy(calibrate[:])
    result = np.empty(shape=(axislen,9))
        
    #lowpass && threhold
    for i in range(0,9,1):   
        for j in range( 0,axislen,1):
            total = 0
                
            if j>N:
                for k in range(N):
                    #if i>=3 and i<=5 and temp[j, i]<0.1 and temp[j, i]>-0.1:
                    #    temp[j, i] =0
                    total += temp[j - k, i]
                result[j, i] =  total/N
                    
            else:
                result[j, i] = 0
                
        
        
    aaaa= result[N+1:axislen,4]
    #print(aaaa)
    lowpassdata =  result[N+1:axislen,:]
    axislen = axislen - N -1
    return lowpassdata

In [5]:
#Signal Magnitude Area
def SMA(acc_data):
    win_size = 20 #window size = 50個點數
    sift_size = int(win_size/2)
    start = 0
    end = win_size
    sma = []
    sma_i = []
    
    for i in range(0,len(acc_data)-win_size,sift_size):
        sma_v = 0
        for j in range(start,end):
            ##IMPORTANT 使用加速度或者角速度 是不一樣的 需要normalize
            sma_v = sma_v + (abs(acc_data[j,3]) + abs(acc_data[j,4]) + abs(acc_data[j,5]))
        sma_v = sma_v / win_size
        
        sma.append(float(sma_v))
        sma_i.append(i)
        
        start = start + sift_size
        end = start + win_size
    #平均值
    avg_line = sum(sma[:]) / len(sma) 
    print("avg line:",avg_line)
    
    start_point = -1
    end_point = -1
    
    print("len(sma):",len(sma))
    for i in range(0,len(sma)):
        if(sma[i] > (avg_line*1.1) and sma[i+1] > (avg_line*1.1) and sma[i+2] > (avg_line*1.1) and start_point == -1 ):
            start_point = sma_i[i]
            break
        #if(sma[i] <(avg_line) and sma[i-1] <(avg_line) and start_point != -1):
        #    end_point = sma_i[i]
    for i in range(len(sma)-1,0,-1):
        if(sma[i] > (avg_line*1.1) and sma[i-1] > (avg_line*1.1) and sma[i-2] > (avg_line*1.1) and end_point == -1 ):
            end_point = sma_i[i]
            break
        if(i<start_point):
            break
            
    if(end_point == -1):
        end_point = len(acc_data)-win_size
        print("not found end point")
    if(start_point == -1):
        start_point = 0
        print("not found start point")
        
    print("start_point",start_point)
    print("end_point",end_point)
    
    """
    #畫平均線
    x = [0,80]
    y = [avg_line,avg_line]
    plt.plot(x,y,"b")

    plt.plot(sma[:])
    plt.show()
    """
    return start_point,end_point
    

In [6]:
# Clawer
def walk_tugt(): 
    home_path = 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\tugt'
    dir_list = []
    for (dirPath, dirNames, fileNames) in os.walk(home_path):
        for f in fileNames :            
            if(f == "TUGT_W"):
                #print(os.sep.join([dirPath, f]))
                dir_list.append(os.sep.join([dirPath, f]))
    return dir_list

def walk_gait(dir_list): 
    home_path = 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait'
    for (dirPath, dirNames, fileNames) in os.walk(home_path):
        for f in fileNames :            
            if(f == "single_L_1" or f == "single_R_1" or f == "single_L_2" or f == "single_R_2"
              or f == "dual_L_1" or f == "dual_R_1" or f == "dual_L_2" or f == "dual_R_2"):
                #print(os.sep.join([dirPath, f]))
                dir_list.append(os.sep.join([dirPath, f]))
    return dir_list

def walk_static(dir_list): 
    home_path = 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\static'
    for (dirPath, dirNames, fileNames) in os.walk(home_path):
        for f in fileNames :            
            dir_list.append(os.sep.join([dirPath, f]))
    return dir_list


In [7]:
# Get all feature
def creatFeatureExcel(lowpassdata,s_p,e_p):
    domainlen = e_p - s_p
    datasize = 6
    # Mean SD VAR MAD RMS
    feature_mean = np.empty(shape=(datasize))
    feature_sd = np.empty(shape=(datasize))
    feature_var = np.empty(shape=(datasize))
    feature_mad = np.empty(shape=(datasize))
    feature_rms = np.empty(shape=(datasize))
    for i in range(0,datasize):
        temp_n = 0
        for j in range(s_p,e_p):
            temp_n = temp_n + lowpassdata[j][i]
        feature_mean[i] = temp_n / domainlen
    print("means:",feature_mean)
    
    for i in range(0,datasize):
        temp=0
        temp2=0
        temp3=0
        for j in range(s_p,e_p):
            diffent = lowpassdata[j,i]-feature_mean[i]
            diffent2 = abs(diffent)
            temp= temp + (diffent*diffent)
            temp2= temp2 + diffent2
            temp3 = temp3 + (lowpassdata[j,i]*lowpassdata[j,i])
        feature_var[i]=temp/(domainlen-1)
        feature_sd[i]=math.sqrt(feature_var[i])
        feature_mad[i]=temp2/(domainlen)
        feature_rms[i]=math.sqrt(temp3/domainlen)
    print("sd:"+str(feature_sd))
    print("var:"+str(feature_var))
    print("mad:"+str(feature_mad))
    print("rms:"+str(feature_rms))
    
    #6 COR
    feature_cor = np.empty(shape=(datasize))
    count=0
    print("cor:")
    for i in range(3):
        temp=0
        for j in range(1,3,1):
            temp = np.corrcoef(lowpassdata[:,i],lowpassdata[:,(i+j)%3])
            feature_cor[count] = temp[0,1]
            print(str(count)+":"+str(feature_cor[count]))
            count=count+1
    #7 IQR
    feature_iqr = np.empty(shape=(6))
    print("iqr:")
    for i in range(6):
        feature_iqr[i] = np.subtract(*np.percentile(lowpassdata[:,i], [75, 25]))   
        print("iqr:"+str(feature_iqr[i]))
        
    #8 SKEW
    feature_skew = np.empty(shape=(6))
    print("skew:")
    for i in range(6):
        feature_skew[i] = scipy.stats.skew(lowpassdata[:,i]) 
        print("skew:"+str(feature_skew[i]))
        
    #9 Kurtosis
    feature_kurtosis = np.empty(shape=(datasize))
    print("kurtosis:")
    for i in range(datasize):
        feature_kurtosis[i] = scipy.stats.kurtosis(lowpassdata[i,:]) 
        print("kurtosis:"+str(feature_kurtosis[i]))
        
    #10 FFT
    feature_fft = np.empty(shape=(6))
    print("fft:")
    for i in range(6):
        temp_fft= np.fft.fft(lowpassdata[:,i]) 
        temp=0
        for j in range(len(temp_fft)):
            temp = temp + abs(temp_fft[j]*temp_fft[j])
        feature_fft[i] = temp / domainlen
        #feature_fft[i] = 0
        print("fft:"+str(feature_fft[i]))
    
    excel_value=(feature_mean.tolist()+feature_sd.tolist()+feature_var.tolist()+feature_mad.tolist()
                +feature_rms.tolist()+feature_cor.tolist()+feature_iqr.tolist()+feature_skew.tolist()
                +feature_kurtosis.tolist()+feature_fft.tolist())
    return excel_value
    

In [8]:
def writeexcel07(path,lable,excel_value):

    wb=Workbook()
    
    excel_name=['ID','label','mean_acc_x','mean_acc_y','mean_acc_z','mean_gry_x','mean_gry_y','mean_gry_z','sd_acc_x','sd_acc_y','sd_acc_z','sd_gry_x','sd_gry_y','sd_gry_z','var_acc_x','var_acc_y','var_acc_z','var_gry_x','var_gry_y','var_gry_z','mad_acc_x','mad_acc_y','mad_acc_z','mad_gry_x','mad_gry_y','mad_gry_z','rms_acc_x','rms_acc_y','rms_acc_z','rms_gry_x','rms_gry_y','rms_gry_z','cor_acc_x','cor_acc_y','cor_acc_z','cor_gry_x','cor_gry_y','cor_gry_z','iqr_acc_x','iqr_acc_y','iqr_acc_z','iqr_gry_x','iqr_gry_y','iqr_gry_z','skew_acc_x','skew_acc_y','skew_acc_z','skew_gry_x','skew_gry_y','skew_gry_z','kurtosis_acc_x','kurtosis_acc_y','kurtosis_acc_z','kurtosis_gry_x','kurtosis_gry_y','kurtosis_gry_z','fft_acc_x','fft_acc_y','fft_acc_z','fft_gry_x','fft_gry_y','fft_gry_z']
        
    sheet=wb.create_sheet(0,"posfeature")
    
    for i in range(0,len(excel_name)):
        sheet.cell(row = 1, column= i+1).value = excel_name[i]
    
    for i in range(0,len(excel_value)):
        sheet.cell(row = i+2,column= 1).value = i
        sheet.cell(row = i+2,column= 2).value = lable[i]
        for j in range(0,len(excel_value[0])):
            sheet.cell(row = i+2, column= j+3).value = excel_value[i][j]

    wb.save(path)

    print("寫入數據成功！")

In [177]:
# MAIN #
file_list = walk_tugt()
file_list = walk_gait(file_list)
file_list = walk_static(file_list)
#print(len(file_list))

"""
file_list = ["C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_L_1"]
file_list.append("C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_L_2")
file_list.append("C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_R_1")
file_list.append("C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_R_2")
"""

features_excel = []
label_excel = []
for f in file_list:
    print("\n")
    s_f = f.split('\\')
    #filepath = "C:\\Users\\ChienChing\\Python 3\\trainingdata\\tugt\\" + s_f[6] + "\\TUGT_W"
    print("fn:",s_f[6],s_f[7])
    
    f = open(f,"rb")
    binarydata = f.read()
    rawdata = np.frombuffer(binarydata,dtype="uint8")
    f.close()

    # rawlen :row data length ,axislen :sample number
    rawlen = len(rawdata)
    axislen=int(rawlen/32)
    axis=np.empty(shape=(axislen,9))
    calibrate=np.empty(shape=(axislen,9))
    lowpassdata=np.empty(shape=(axislen,9))
    
    #print("axislen",axislen)
    
    axis = setaxis(rawdata)
    calibrate = calibrateData_v1(axis,axislen)
    lowpassdata = lowpass(calibrate,axislen)
    start_p,end_p = SMA(lowpassdata)
    #plot_graph_byname(lowpassdata[:,4],start_p,end_p,(s_f[6]+"_"+s_f[7]))
    label_excel.append(s_f[5])
    features_excel.append(creatFeatureExcel(lowpassdata,start_p,end_p))
    
save_path = "C:\\Users\\ChienChing\\Python 3\\trainingdata.xlsx"       
writeexcel07(save_path,label_excel,features_excel)  
#print(features_excel)


    #plot_graph(lowpassdata[:,4],0,1)
    
    #print("len:",len(calibrate))




fn: S2014_0905_1 TUGT_W
==setaxis==
==calibrateData==
==lowpass==
avg line: 41.332854568318325
len(sma): 200
not found end point
start_point 240
end_point 1998
means: [-0.23185698 -0.18600858  0.9031276  -0.84468669 -1.39470933  0.48997054]
sd:[  0.22349336   0.08579463   0.07211186   6.33466396  24.59564537
  35.17369839]
var:[  4.99492842e-02   7.36071780e-03   5.20011975e-03   4.01279674e+01
   6.04945771e+02   1.23718906e+03]
mad:[  0.15617569   0.07715885   0.05111792   5.2348757   15.29185933
  24.90829121]
rms:[  0.32199151   0.20483096   0.90600034   6.38894648  24.62817238
  35.16710652]
cor:
0:-0.399407107898
1:0.48935349009
2:-0.215661528164
3:-0.399407107898
4:0.48935349009
5:-0.215661528164
iqr:
iqr:0.14462674
iqr:0.13472598
iqr:0.070468245
iqr:8.18071321321
iqr:14.0103003003
iqr:25.8443018018
skew:
skew:-1.69942577982
skew:0.1996161691
skew:-1.14074288681
skew:-0.122357373338
skew:0.516445238068
skew:0.101350718691
kurtosis:
kurtosis:-0.690621624655
kurtosis:-0.62032322

In [173]:
# Plot graph
def plot_graph(y,s,e):
    x = np.arange(0,len(y)*0.01,0.01)
    plt.plot(x,y)


    plt.plot(int(s)*0.01,0,"ro")
    plt.plot(int(e)*0.01,0,"yo")
    #plt.savefig("C:\\Users\\ChienChing\\Python 3\\pic\\aa.png")
    plt.show()
def plot_graph_byname(y,s,e,filename):
    x = np.arange(0,len(y)*0.01,0.01)
    
    if(not(len(x)==len(y))):
        x = x[0:len(x)-1]
    print("len x,y:",len(x),len(y))
    plt.plot(x,y)

    plt.plot([int(s)*0.01,int(s)*0.01],[100,-100],"r")
    plt.plot([int(e)*0.01,int(e)*0.01],[100,-100],"y")
    plt.savefig("C:\\Users\\ChienChing\\Python 3\\pic\\" + filename + ".png")
    #plt.show()
    plt.cla()
    

In [10]:
aa = [[1,"aa"],[2,"bb"]]
cc = np.array(aa)
print(cc[:,0])
print(cc[0])
#plt.plot(aa[:][0], 'ro')
#plt.axis([0, 6, 0, 20])
#plt.show()

['1' '2']
['1' 'aa']


In [137]:
print(walk_gait())

['C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_L_1', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_L_2', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_R_1', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\dual_R_2', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\single_L_1', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\single_L_2', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\single_R_1', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0905_2\\single_R_2', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0910_1\\dual_L_1', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0910_1\\dual_L_2', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0910_1\\single_L_1', 'C:\\Users\\ChienChing\\Python 3\\trainingdata\\gait\\S2014_0910_1\\single_L_2', 'C:\\Users\\ChienChing\\Python 3\\train

In [12]:
aaa = ["a_a","b_b"]
bbb = aaa[0].split('_')[0]
print(bbb)

a


In [13]:

"""
#main
filepath = "C:\\Users\\ChienChing\Python 3\\trainingdata\\tugt\\S2014_0905_2\\TUGT_W"
f = open(filepath,"rb")
binarydata = f.read()
rawdata = np.frombuffer(binarydata,dtype="uint8")
f.close()

# rawlen :row data length ,axislen :sample number
rawlen = len(rawdata)
axislen=int(rawlen/32)
axis=np.empty(shape=(axislen,9))
calibrate=np.empty(shape=(axislen,9))
lowpassdata=np.empty(shape=(axislen,9))

#print(axislen)

axis = setaxis(rawdata)

calibrate = calibrateData_v1(axis,axislen)

lowpassdata = lowpass(calibrate,axislen)

start_p,end_p = SMA(lowpassdata)

print(lowpassdata)

plot_graph(lowpassdata[:,4],start_p,end_p)
#plot_graph(calibrate[:,4],start_p,end_p)
"""

'\n#main\nfilepath = "C:\\Users\\ChienChing\\Python 3\\trainingdata\\tugt\\S2014_0905_2\\TUGT_W"\nf = open(filepath,"rb")\nbinarydata = f.read()\nrawdata = np.frombuffer(binarydata,dtype="uint8")\nf.close()\n\n# rawlen :row data length ,axislen :sample number\nrawlen = len(rawdata)\naxislen=int(rawlen/32)\naxis=np.empty(shape=(axislen,9))\ncalibrate=np.empty(shape=(axislen,9))\nlowpassdata=np.empty(shape=(axislen,9))\n\n#print(axislen)\n\naxis = setaxis(rawdata)\n\ncalibrate = calibrateData_v1(axis,axislen)\n\nlowpassdata = lowpass(calibrate,axislen)\n\nstart_p,end_p = SMA(lowpassdata)\n\nprint(lowpassdata)\n\nplot_graph(lowpassdata[:,4],start_p,end_p)\n#plot_graph(calibrate[:,4],start_p,end_p)\n'